# Mauritania country profile

In [1]:
import os, sys, glob, json
from itertools import product, compress, chain
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf, save_npz, load_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_GrainTypeAgg, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools import FDW_PD_CaliSeasonYear
from tools_graphic import PlotBarProduction, PlotLinePAY, PlotHeatCropSystem, PlotHeatSeasonData
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# CPCV2 grain code ------------------------------ #
grain_code = pd.read_hdf('../data/crop/grain_cpcv2_code.hdf')
product_category = grain_code[['product', 'product_category']].set_index('product').to_dict()['product_category']
# ----------------------------------------------- #

# Load FEWS NET administrative boundaries ------- #
epsg = 'epsg:3243' # Mauritania
fn_shapes = sorted(glob.glob('../data/shapefile/fewsnet/MR_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
parameters = {
    'format': 'json',
    'country': 'Mauritania',
    'product': ['R011','R012','R013','R014','R015','R016','R017','R018','R019'],
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
df_origin = df.copy()
# ----------------------------------------------- #

- FEWS NET admin shapefiles ------------------- #
| year	 | Admin1   | # units   | Admin2   | # units   | Admin3   | # units   |
| 1990	 | MR1990A1 | 13	| MR1990A2	| 45	| nan	| 0	|
----------------------------------------------- #


In [3]:
# Manual Pre-processing before Sweeping --------- #
# 1. Default setting
# a) None-type crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'none'
df.loc[df['crop_production_system'] == '', 'crop_production_system'] = 'none'
# b) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'

# 2. Manual setting
# a) Add admin names to reporting units
admin_level = 'admin_1'
reporting_unit = df.loc[df['fnid'].map(lambda x: x[6] != 'A'), 'fnid'].unique()
df.loc[df.fnid.isin(reporting_unit),admin_level] = df.loc[df.fnid.isin(reporting_unit), 'locality_name'].map(lambda x: x.split(',')[0])

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df)
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([MR_Admin1_1990, MR_Admin2_1990], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- Remove missing records ---------------------- #
Orignial data points: 4,084
Removed 1,329 "Missing Value" points
0/1,021 "Area Harvested" points are retained.
909/1,021 "Area Planted" points are retained.
979/1,021 "Quantity Produced" points are retained.
867/1,021 "Yield" points are retained.
Current data points: 2,755

- Minor changes are applied ------------------- #

- Basic information --------------------------- #
Data period: 1989 - 2017
7 grain types are found: Cowpeas (Mixed), Groundnuts (In Shell), Maize (Corn), Millet, Rice (Paddy), Sorghum, Wheat Grain
7 seasons are found: Annual (06-01), Bas-fond (10-01), Dam retention (10-01), Main (06-01), Walo (11-01), Hot off-season (03-01), Cold off-season (11-01)
7 crop production system are found: none, surface water, dam irrigation, dieri, flood recessional , parastatal recessional , irrigated
Data sources include:
[1] Ministry of Rural Development, Mauritania --- FEWS NET Agro Maps, Mauritania
[2] Ministry of Rural Development, 

## Mauritania
Mauritania crop seasonal calendar

![FEWS NET](https://fews.net/sites/default/files/styles/large_width_880/public/2023-03/seasonal-calendar-mauritania.png?itok=fcsQC56T)

![USDA](https://ipad.fas.usda.gov/rssiws/al/crop_calendar/images/wafrica_mr_calendar.png)

- comparison of boundaries
![image](../figures/MR_admin_shapes.png)

- FDW data consists of `MR1990A1`.

- FEWS NET admin shapefiles ------------------- #

| year	 | Admin1   | # units   | Admin2   | # units   | Admin3   | # units   |
| :---: | :----:  | :----:   | :----:   | :---:  | :---:  | :---:  |
| 1990	 | MR1990A1 | 13	| MR1990A2	| 45	| nan	| 0	|

- **`MR1990A1`** is used to represent the current admin-level 1 crop data.
- Mauritania has seven crop seasons: `Annual`, `Dam retention`, `Bas-fond`, `Main`, `Walo`, `Hot off-season`, `Cold off-season`.
- Mauritania has seven crop production systems: `none`, `dam irrigation`, `surface water`, `dieri`, `flood recessional` , `parastatal recessional` , `irrigated`.


In [4]:
# Define the latest shapefile ------------------- #
latest_level = 1
shape_latest = MR_Admin1_1990.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
mdx_pss = area.columns.droplevel([0,1]).unique()
# ----------------------------------------------- #

In [5]:
# Link admin boundaries ------------------------- #
link, over = FDW_PD_CreateAdminLink(MR_Admin1_1990, MR_Admin1_1990, 'ADMIN1', 'ADMIN1', prod, epsg)
# Crop specific ratios
link_ratio = FDW_PD_RatioAdminLink(link, prod, over, mdx_pss)
# Add current unit to link_ratio
for fnid_new in link_ratio.keys():
    link_ratio[fnid_new][fnid_new] = 1.0
    link_ratio[fnid_new] = link_ratio[fnid_new].sort_index(axis=1, ascending=False)
# Connect data with AdminLink
area_new, prod_new = FDW_PD_ConnectAdminLink(link_ratio, area, prod, validation=False)
# ----------------------------------------------- #

CBR is considered for 'MR1990A108' as no record found in: ['MR1990A108']
CBR is considered for 'MR1990A113' as no record found in: ['MR1990A113']
CBR is considered for 'MR1990A111' as no record found in: ['MR1990A111']


In [6]:
# Manual correction ----------------------------- #
crop_new = prod_new/area_new

# Complete long format DataFrame ---------------- #
df_area = area_new.T.stack().rename('value').reset_index()
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().rename('value').reset_index()
df_prod['indicator'] = 'production'
df_yield = (prod_new/area_new).T.stack().rename('value').reset_index()
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Add "planting year"
cols = ['season_name','product','crop_production_system','planting_month','harvest_year','harvest_month','planting_year']
season_table = df[cols].drop_duplicates()
stack = stack.merge(season_table, on=cols[:-1])
# Add country and admin names
stack = stack.merge(df[['fnid','country','country_code','admin_1','admin_2']].drop_duplicates(), on='fnid', how='inner')
names = [
    'fnid','country','country_code','admin_1','admin_2','name',
    'product','season_name','planting_year','planting_month','harvest_year','harvest_month',
    'crop_production_system','indicator','value'
]
stack = stack[names]
# ----------------------------------------------- #

# Final Processing ------------------------------ #
# Manual change of product name
product_category_custom = {
    'Groundnuts (In Shell)': 'Groundnuts (In Shell)',
    'Cowpeas (Mixed)':'Cowpeas',
    'Maize (Corn)': 'Maize',
    'Millet': 'Millet',
    'Rice (Paddy)': 'Rice',
    'Sorghum': 'Sorghum',
    'Wheat Grain': 'Wheat'
}

stack['product'] = stack['product'].replace(product_category_custom)
for f, r in link_ratio.items():
    mdx = pd.MultiIndex.from_frame(r.index.to_frame().reset_index(drop=True).replace(product_category_custom))
    r.index = mdx
    link_ratio[f] = r
# Calibration of crop calendar using External Crop Calendar (ECC)
ecc = pd.read_csv('../data/crop_calendar/external_crop_calendar.csv')
ecc = ecc[ecc['country'] == 'Mauritania']
stack, link_ratio = FDW_PD_CaliSeasonYear(stack, ecc, link_ratio)

########################################################################
# None admin names
stack.loc[stack['admin_1'].isna(), 'admin_1'] = 'none'
stack.loc[stack['admin_2'].isna(), 'admin_2'] = 'none'
# ----------------------------------------------- #

# Save data
fn_out = '../data/crop/adm_crop_production_MR.csv'
stack.to_csv(fn_out); print(f'{fn_out} is saved.')
save_npz('../data/crop/adm_crop_production_MR_ratio.npz', link_ratio)

Below data are not in external crop calendar:
       country season_name  product   crop_production_system     _merge
21  Mauritania        Walo    Maize       flood recessional   left_only
23  Mauritania        Walo    Maize  parastatal recessional   left_only
24  Mauritania        Walo     Rice       flood recessional   left_only
25  Mauritania        Walo  Sorghum       flood recessional   left_only
27  Mauritania        Walo  Sorghum  parastatal recessional   left_only


ValueError: Some data are not in the external crop calendar. Please check the data.

In [7]:
stack[['country','season_name','product','crop_production_system','planting_month','harvest_month']].drop_duplicates().sort_values(['country','season_name','product','crop_production_system']).reset_index(drop=True)

,country,season_name,product,crop_production_system,planting_month,harvest_month
0,Mauritania,Annual,Cowpeas,none,06-01,05-01
1,Mauritania,Annual,Groundnuts (In Shell),none,06-01,05-01
2,Mauritania,Annual,Maize,none,06-01,05-01
3,Mauritania,Annual,Millet,none,06-01,05-01
4,Mauritania,Annual,Sorghum,none,06-01,05-01
5,Mauritania,Bas-fond,Maize,surface water,10-01,03-01
6,Mauritania,Bas-fond,Rice,surface water,10-01,03-01
7,Mauritania,Bas-fond,Sorghum,surface water,10-01,03-01
8,Mauritania,Bas-fond,Wheat,surface water,10-01,03-01
9,Mauritania,Cold off-season,Wheat,irrigated,11-01,02-01


In [ ]:
# Bar chart of national grain production
country_iso, country_name = 'MR', 'Mauritania'
df = pd.read_hdf('../data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
year = [1995, df['year'].max()]
product_order = ['Sorghum','Maize','Rice','Millet','Wheat']
for season_name in ['Annual', 'Dam retention', 'Bas-fond', 'Main', 'Walo', 'Hot off-season', 'Cold off-season']:
    footnote = 'National grain production in %s - %s' % (country_name, season_name)
    fn_save = '../figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, year, product_order, footnote, fn_save)
    fig.show()


In [ ]:
# Lineplot of Production-Area-Yield (PAY) time-series
country_iso, country_name = 'MR', 'Mauritania'
df = pd.read_hdf('../data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_season = [
    ['Maize','Main'],
    ['Maize','Annual'],
    ['Maize','Bas-fond'],
    ['Maize','Dam retention'],
    ['Maize', 'Walo'],
    ['Millet','Annual'],
    ['Millet','Main'],
    ['Rice','Walo'],
    ['Rice','Main'],
    ['Rice','Bas-fond'],
    ['Rice','Dam retention'],
    ['Rice','Hot off-season'],
    ['Sorghum','Walo'],
    ['Sorghum','Main'],
    ['Sorghum','Bas-fond'],
    ['Sorghum','Dam retention'],
    ['Sorghum','Annual'],
    ['Wheat','Bas-fond'],
    ['Wheat','Cold off-season'],
    ['Wheat','Dam retention']
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield (PAY) time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = '../figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, year, footnote, fn_save)
    fig.show()

In [ ]:
# Calibrated PAY time-series per FNID
from tools_graphic import PlotLineCropTS
country_iso, country_name = 'MR', 'Mauritania'
df = pd.read_hdf('../data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
link_ratio = load_npz('../data/crop/adm_crop_production_%s_ratio.npz' % country_iso)
year_all = np.arange(df['year'].min(), df['year'].max()+1)
prod_season = [
    ['Maize', 'Main'],
    ['Maize', 'Annual'],
    ['Maize', 'Bas-fond'],
    ['Maize', 'Dam retention'],
    ['Maize', 'Walo'],
    ['Millet', 'Annual'],
    ['Millet', 'Main'],
    ['Rice', 'Walo'],
    ['Rice', 'Main'],
    ['Rice', 'Bas-fond'],
    ['Rice', 'Dam retention'],
    ['Rice', 'Hot off-season '],
    ['Sorghum', 'Walo'],
    ['Sorghum', 'Main'],
    ['Sorghum', 'Bas-fond'],
    ['Sorghum', 'Dam retention'],
    ['Sorghum', 'Annual'],
    ['Wheat', 'Bas-fond'],
    ['Wheat', 'Cold off-season'],
    ['Wheat', 'Dam retention']
]
for product_name, season_name in prod_season:
    sub = df[
        (df['product'] == product_name) &
        (df['season_name'] == season_name)
    ]
    for fnid in sub['fnid'].unique():
        sub_fps = sub[sub['fnid'] == fnid]
        fn_save = '../figures/crop_calibrated/%s_%s_%s_%s.png' % (country_iso, product_name, season_name, fnid)
        fig = PlotLineCropTS(sub_fps, fnid, product_name, season_name, link_ratio, year_all, fn_save)

